# Working with POSTGIS

In this chapter we will cover the following topics:

<ul>
    <li>Executing a PostGIS ST_Buffer analysis query and exporting it to GeoJSON</li>
    <li>Finding out whether a point is inside a polygon</li>
    <li>Splitting LineStrings at intersections using ST_Node</li>
    <li>Checking the validity of LineStrings</li>
    <li>Executing a spatial join and assigning point attributes to a polygon</li>
    <li>Conducting a complex spatial analysis query using ST_Distance()</li>
    </ul>

## Introduction

A spatial database is nothing but a standard database that can store geometry and execute spatial queries in their simplest forms. We will explore how to run spatial analysis queries, handle connections, and more, all from our Python code. Your ability to answer spatial questions such as "I want to locate all the hotels that are within 2 km of a golf course and less than 5 km from a park" is where PostGIS comes into play. This chaining of requests into a model is where the powers of spatial analysis shine.

We will work with the most popular and powerful open source spatial database called <strong>PostgreSQL</strong>, along with the <strong>PostGIS</strong> extension, including over 150 functions. Basically, we'll get a full-blown GIS with complex spatial analysis functions for both vectors and rasters, spatial data types, and diverse methods to move spatial data around.

If you are looking for more information on PostGIS and a good read, please check out PostGIS Cookbook by Paolo Corti (available at https://www.packtpub.com/big-data-and-business-intelligence/postgis-cookbook). This book explores the wider use of PostGIS and includes a full chapter on PostGIS Programming using Python.


## Executing a PostGIS ST_BUFFER Analysis Query and exporting it to GeoJSON

Let's start by executing our first spatial analysis query from Python against our already running PostgreSQL and PostGIS database. The goal is to generate a 100 m buffer around all schools and export the new buffer polygon to GeoJSON, including the name of a school. The end result will be shown on this map, available (https://github.com/mdiener21/python-geospatial-analysis-cookbook/blob/master/ch04/geodata/out_buff_100m.geojson) on GitHub.

<img src="./50790OS_04_01.jpg" height=400 width=400>

To get started, we'll use our data in the PostGIS database. We will begin by accessing our schools table that we uploaded to PostGIS in the Batch importing a folder of Shapefiles into PostGIS using ogr2ogr recipe of Chapter 3, Moving Spatial Data from One Format to Another.

Connecting to a PostgreSQL and PostGIS database is accomplished with Psycopg, which is a Python DB API (http://initd.org/psycopg/) implementation. We've already installed this in Chapter 1, Setting Up Your Geospatial Python Environment along with PostgreSQL, Django, and PostGIS.

In [ ]:
#!/usr/bin/env python

import psycopg2
import json
from geojson import loads, Feature, FeatureCollection

# Database connection information
db_host = "localhost"
db_user = "calvin"
db_passwd = "planets"
db_database = "py_test"
db_port = "5432"

# connect to database
conn = psycopg2.connect(host=db_host, user=db_user,
        port=db_port, password=db_passwd, database=db_database)

# create a cursor
cur = conn.cursor()

# the PostGIS buffer query
buffer_query = """SELECT ST_AsGeoJSON(ST_Transform(
        ST_Buffer(wkb_geometry, 100,'quad_segs=8'),4326)) 
        AS geom, name
        FROM geodata.schools"""

# execute the query
cur.execute(buffer_query)

# return all the rows, we expect more than one
dbRows = cur.fetchall()

# an empty list to hold each feature of our feature collection
new_geom_collection = []

# loop through each row in result query set and add to my feature collection
# assign name field to the GeoJSON properties
for each_poly in dbRows:
    geom = each_poly[0]
    name = each_poly[1]
    geoj_geom = loads(geom)
    myfeat = Feature(geometry=geoj_geom, properties={'name': name})
    new_geom_collection.append(myfeat)

# use the geojson module to create the final Feature Collection of features created from for loop above
my_geojson = FeatureCollection(new_geom_collection)

# define the output folder and GeoJSon file name
output_geojson_buf = "../geodata/out_buff_100m.geojson"


# save geojson to a file in our geodata folder
def write_geojson():
    fo = open(output_geojson_buf, "w")
    fo.write(json.dumps(my_geojson))
    fo.close()

# run the write function to actually create the GeoJSON file
write_geojson()

# close cursor
cur.close()

# close connection
conn.close()

## How it works

The database connection is using the pyscopg2 module, so we import the libraries at the start alongside geojson and the standard json modules to handle our GeoJSON export.

Our connection is created and then followed immediately with our SQL Buffer query string. The query uses three PostGIS functions. Working your way from the inside out, you will see the ST_Buffer function taking in the geometry of the school points followed by the 100 m buffer distance and the number of circle segments that we would like to generate. ST_Transform then takes the newly created buffer geometry and transforms it into the WGS84 coordinate system (EPSG: 4326) so that we can display it on GitHub, which only displays WGS84 and the projected GeoJSON. Lastly, we'll use the ST_asGeoJSON function to export our geometry as the GeoJSON geometry.

<pre>
<strong>Note:</strong>

PostGIS does not export the complete GeoJSON syntax, only the geometry in the form of the GeoJSON geometry. This is the reason that we need to complete our GeoJSON using the Python geojson module.
</pre>

All of this means that we not only perform analysis on the query, but we also specify the output format and coordinate system all in one go.

Next, we will execute the query and fetch all the returned objects using cur.fetchall() so that we can later loop through each returned buffer polygon. Our new_geom_collection list will store each of the new geometries and the feature names. Next, in the for loop function, we'll use the geojson module function, loads(geom), to input our geometry into a GeoJSON geometry object. This is followed by the Feature()function that actually creates our GeoJSON feature. This is then used as the input for the FeatureCollection function where the final, completed GeoJSON is created.

Lastly, we'll need to write this new GeoJSON file to disk and save it. Hence, we'll use the new file object where we use the standard Python json.dumps module to export our FeatureCollection.

We'll do a little clean up to close the cursor object and connection. Bingo! We are now done and can visualize our final results.

## Finding out whether a point is inside a polygon

A point inside a polygon analysis query is a very common spatial operation. This query can identify objects located within an area such as a polygon. The area of interest in this example is a 100 m buffer polygon around bike paths and we would like to locate all schools that are inside this polygon.

### Getting ready

In the previous section, we used the schools table to create a buffer. This time around, we will use this table as our input points table. The bikeways table that we imported in Chapter 3, Moving Spatial Data from One Format to Another, will be used as our input lines to generate a new 100 m buffer polygon. Be sure, however, that you have the two datasets in your local PostgreSQL database.

### How to do it
Now, let's dive into some more code to find schools located within 100 m of the bikeways in order to find points inside a polygon:


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import json
import psycopg2
from geojson import loads, Feature, FeatureCollection


# Database Connection Info
db_host = "localhost"
db_user = "pluto"
db_passwd = "stars"
db_database = "py_geoan_cb"
db_port = "5432"

# connect to DB
conn = psycopg2.connect(host=db_host, user=db_user, port=db_port, password=db_passwd, database=db_database)

# create a cursor
cur = conn.cursor()

# uncomment if needed
# cur.execute("Drop table if exists geodata.bikepath_100m_buff;")

# query to create a new polygon 100m around the bikepath
new_bike_buff_100m = """ CREATE TABLE geodata.bikepath_100m_buff 
       AS SELECT name, 
       ST_Buffer(wkb_geometry, 100) AS geom
       FROM geodata.bikeways; """

# run the query
cur.execute(new_bike_buff_100m)

# commit query to database
conn.commit()

# query to select schools inside the polygon and output geojson
is_inside_query = """ SELECT s.name AS name, 
    ST_AsGeoJSON(ST_Transform(s.wkb_geometry,4326)) AS geom
    FROM geodata.schools AS s,
    geodata.bikepath_100m_buff AS bp
        WHERE ST_WITHIN(s.wkb_geometry, bp.geom); """

# execute the query
cur.execute(is_inside_query)

# return all the rows, we expect more than one
db_rows = cur.fetchall()

# an empty list to hold each feature of our feature collection
new_geom_collection = []

def export2geojson(query_result):
    """
    loop through each row in result query set and add to my feature collection
    assign name field to the GeoJSON properties
    :param query_result: pg query set of geometries
    :return: new geojson file
    """

    for row in db_rows:
        name = row[0]
        geom = row[1]
        geoj_geom = loads(geom)
        myfeat = Feature(geometry=geoj_geom, 
                    properties={'name': name})
        new_geom_collection.append(myfeat)

    # use the geojson module to create the final Feature
    # Collection of features created from for loop above
    my_geojson = FeatureCollection(new_geom_collection)
    # define the output folder and GeoJSon file name
    output_geojson_buf = "../geodata/out_schools_in_100m.geojson"

    # save geojson to a file in our geodata folder
    def write_geojson():
        fo = open(output_geojson_buf, "w")
        fo.write(json.dumps(my_geojson))
        fo.close()

    # run the write function to actually create the GeoJSON file
    write_geojson()


export2geojson(db_rows)

You can now view your newly created GeoJSON file on a great little site created by Mapbox at http://www.geojson.io. Simply drag and drop your GeoJSON file from Windows Explorer in Windows or Nautilus in Ubuntu onto the http://www.geojson.io web page and, Bob's your uncle, you should see 50 or so schools that are located within 100 m of a bikeway in Vancouver.

<img src="./50790OS_04_02.jpg" height=400 width=400>

### How it works
We will reuse code to make our database connection, so this should be familiar to you at this point. The new_bike_buff_100m query string contains our query to generate a new 100 m buffer polygon around all the bikeways. We need to execute this query and commit it to the database so that we can access this new set of polygons as input to our actual query that will find schools (points) located inside this new buffer polygon.

The is_inside_query string actually does the hard work for us by selecting selecting the values from the field name and the geometry from the geom field. The geometry is wrapped up in two other PostGIS functions to allow us to export our data as GeoJSON in the WGS 84 coordinate system. This will be the input geometry needed to generate our final new GeoJSON file.

The WHERE clause uses the ST_Within function to see whether a point is inside the polygon and returns True if the point is within the buffer polygon.

Now, we've created a new function that simply wraps up our export to the GeoJSON code that was used in the previous, Executing a PostGIS ST_Buffer analysis query and exporting it to GeoJSON, recipe. This new export2geojson function simply takes one input of our PostGIS query and outputs a GeoJSON file. To set the name and location of the new output file, simply replace the path and name within the function.

Finally, all we need to do is call the new function to export the GeoJSON file using the db_rows variable that contains our list of schools as points located within the 100 m buffer polygon.

### There's more...

This example to find all schools located within 100 m of the bike paths could be completed using another PostGIS function called ST_Dwithin.

The SQL to select all the schools located within 100 m of the bikepaths would look like this:

<code>SELECT *  FROM geodata.bikeways as b, geodata.schools as s where ST_DWithin(b.wkb_geometry, s.wkb_geometry, 100)
</code>

## Splitting LineStrings at Intersections using ST_Node

Working with road data is usually a tricky business because the validity of the data and data structure plays a very important role. If you want to do anything useful with your road data, such as building a routing network, you will need to prepare the data first. The first task is usually to segmentize your lines, which means splitting all lines at intersections where LineStrings cross each other, creating a base network road dataset.

<pre>
<strong>Note</strong>

Be aware that this recipe will split all lines on all intersections regardless of whether, for example, there is a road-bridge overpass where no intersection should be created.
</pre>

### Getting Ready

Before we get into the details of how to do this, we will use a small section of the OpenStreetMap (OSM) road data for our example. The OSM data is available in your /ch04/geodata/folder called vancouver-osm-data.osm. This data was simply downloaded from the www.openstreetmap.org home page using the Export button located at the top of the page:

<img src="./50790OS_04_03.jpg" height=400 width=500>

The OSM data contains not only roads but all the other points and polygons located within the extent that I have chosen. The region of interest is again the Burrard Street bridge in Vancouver.

We are going to need to extract all the roads and import them into our PostGIS table. This time, let's try using the ogr2ogr command line directly from the console to upload the OSM streets to our PostGIS database:

<code>ogr2ogr -lco SCHEMA=geodata -nlt LINESTRING -f "PostgreSQL" PG:"host=localhost port=5432 user=pluto dbname=py_geoan_cb password=stars" ../geodata/vancouver-osm-data.osm lines -t_srs EPSG:3857
</code>

This assumes that your OSM data is in the /ch04/geodata folder and the command is run while you are located in the /ch04/code folder.

Now this really long thing means that we connect to our PostGIS database as our output and input the vancouver-osm-data.osm file. Create a new table called lines and transform the input OSM projection to EPSG:3857. All data exported from OSM is in EPSG:4326. You can, of course, leave it in this system and simply remove the -t_srs EPSG:3857 part of the command line option.

We are now ready to rock and roll with the splitting at intersections. If you like, go ahead and open the data in QGIS (Quantum GIS). In QGIS, you will see that the road data is not split at all road intersections as shown in this screenshot:

<img src="./50790OS_04_04.jpg" height=400 width=500>

Here, you can see that McNicoll Avenue is a single LineString crossing over Cypress Street. After we've completed the recipe, we will see that McNicoll Avenue will be split at this intersection.

In [ ]:
#!/usr/bin/env python

import psycopg2
import json
from geojson import loads, Feature, FeatureCollection

# Database Connection Info
db_host = "localhost"
db_user = "pluto"
db_passwd = "stars"
db_database = "py_geoan_cb"
db_port = "5432"

# connect to DB
conn = psycopg2.connect(host=db_host, user=db_user, 
    port=db_port, password=db_passwd, database=db_database)

# create a cursor
cur = conn.cursor()

# drop table if exists
# cur.execute("DROP TABLE IF EXISTS geodata.split_roads;")

# split lines at intersections query
split_lines_query = """
 CREATE TABLE geodata.split_roads
    (ST_Node(ST_Collect(wkb_geometry)))).geom AS geom
    FROM geodata.lines;"""

cur.execute(split_lines_query)
conn.commit()

cur.execute("ALTER TABLE geodata.split_roads ADD COLUMN id serial;")
cur.execute("ALTER TABLE geodata.split_roads ADD CONSTRAINT split_roads_pkey PRIMARY KEY (id);")

# close cursor
cur.close()

# close connection
conn.close()

<img src="./50790OS_04_05.jpg" height=400 width=500>

Well, this was quite simple and we can now see that McNicoll Avenue is split at the intersection with Cypress Street.

### HOw it works

Looking at the code, we can see that the database connection remains the same and the only new thing is the query itself that creates the intersection. Here three separate PostGIS functions are used to obtain our results:

<ul>
    <li>The first function, when working our way inside-out in the query, starts with ST_Collect(wkb_geometry). This simply takes our original geometry column as input. The simple combining of the geometries is all that is going on here.</li>
    <li>Next up is the actual splitting of the lines using the ST_Node(geometry), inputting the new geometry collection and nodding, which splits our LineStrings at intersections.</li>
    <li>Finally, we'll use ST_Dump() as a set returning function. This means that it basically explodes all the LineString geometry collections into individual LineStrings. The end of the query with .geom specifies that we only want to export the geometry and not the returned array numbers of the split geometry.</li> </ul>

Now, we'll execute and commit the query to the database. The commit is an important part because, otherwise, the query will be run but it will not actually create the new table that we are looking to generate. Last but not least, we can close down our cursor and connection. That is that; we now have split LineStrings.

<pre>
<strong>Note:</strong>

Be aware that the new split LineStrings do NOT contain the street names and other attributes. To export the names, we would need to do a join on our data. Such a query to include the attributes on the newly created LineStrings could look like this:

<code>
CREATE TABLE geodata.split_roads_attributes AS SELECT
   r.geom,
   li.name,
   li.highway
FROM
   geodata.lines li,
   geodata.split_roads r
WHERE
   ST_CoveredBy(r.geom, li.wkb_geometry)
</code>
</pre>


## Checking the Validity of LineStrings

Working with road data has many areas to watch out for and one of these is invalid geometry. Our source data is OSM and is, therefore, collected by a community of users that are not trained by GIS professionals, resulting in errors. To execute spatial queries, the data must be valid or we will have results with errors or no results at all.

PostGIS includes the ST_isValid() function that returns True/False on the basis of whether a geometry is valid or not. There is also the ST_isValidReason() function that will output a text description of the geometry error. Finally, the ST_isValidDetail() function will return if the geometry is valid along with the reason and location of the geometry error. These three functions all accomplish similar tasks and selecting one depends on what you want to accomplish.

### How to do it...

1. Now, to determine if geodata.lines are valid, we will run another query that will list all invalid geometries if there are any:

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import psycopg2

# Database Connection Info
db_host = "localhost"
db_user = "pluto"
db_passwd = "stars"
db_database = "py_geoan_cb"
db_port = "5432"

# connect to DB
conn = psycopg2.connect(host=db_host, user=db_user, 
    port=db_port, password=db_passwd, database=db_database)

# create a cursor
cur = conn.cursor()

# the PostGIS buffer query
valid_query = """SELECT
                   ogc_fid, 
                   ST_IsValidDetail(wkb_geometry)
                FROM 
                   geodata.lines
                WHERE NOT
                   ST_IsValid(wkb_geometry);
                """

# execute the query
cur.execute(valid_query)

# return all the rows, we expect more than one
validity_results = cur.fetchall()

print validity_results

# close cursor
cur.close()

# close connection
conn.close();

This query should return an empty Python list, which means that we have no invalid geometries. If there are objects in your list, then you'll know that you have some manual work to do to correct those geometries. Your best bet is to fire up QGIS and get started with digitizing tools to clean things up.

## Executing a spatial join and assigning point attributes to a polygon

We'll now get back to some more golf action where we would like to execute a spatial attribute join. We're given a situation where we have a set of polygons, in this case, these are in the form of golf greens without any hole number. Our hole number is stored in a point dataset that is located spatially within the green of each hole. We would like to assign each green its appropriate hole number based on its location within the polygon.

The OSM data from the Pebble Beach Golf Course located in Monterey California is our source data. This golf course is one the great golf courses on the PGA tour and is well mapped in OSM.

### Getting ready

Importing our data into PostGIS will be the first step to execute our spatial query. This time around, we will use the shp2pgsql tool to import our data to change things a little since there are so many ways to get data into PostGIS. The shp2pgsql tool is definitely the most well-tested and common way to import Shapefiles into PostGIS. Let's get going and perform this import once again, executing this tool directly from the command line.

For Windows users, this should work, but check that the paths are correct or that shp2pgsql.exe is in your system path variable. By doing this, you save having to type the full path to execute.

e.g.

<code>shp2pgsql -s 4326 ..\geodata\shp\pebble-beach-ply-greens.shp geodata.pebble_beach_greens | psql -h localhost -d py_geoan_cb -p 5432 -U pluto
</code>

On a Linux machine your command is basically the same without the long path, assuming that your system links were all set up when you installed PostGIS in Chapter 1, Setting Up Your Geospatial Python Environment.

Next up, we need to import our points with the attributes, so let's get to it as follows:

<code>shp2pgsql -s 4326 ..\geodata\shp\pebble-beach-pts-hole-num-green.shp geodata.pebble_bea-ch_hole_num | psql -h localhost -d py_geoan_cb -p 5432 -U postgres
</code>

That's that! We now have our points and polygons available in the PostGIS Schema geodata setting, which sets the stage for our spatial join.

### How to do it

The core work is done once again inside our PostGIS query string, assigning the attributes to the polygons, so follow along:

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import psycopg2

# Database Connection Info
db_host = "localhost"
db_user = "pluto"
db_passwd = "stars"
db_database = "py_geoan_cb"
db_port = "5432"

# connect to DB
conn = psycopg2.connect(host=db_host, user=db_user, port=db_port, password=db_passwd, database=db_database)

# create a cursor
cur = conn.cursor()

# assign polygon attributes from points
spatial_join = """  UPDATE geodata.pebble_beach_greens AS g 
                        SET 
                           name = h.name
                        FROM 
                           geodata.pebble_beach_hole_num AS h
                        WHERE 
                           ST_Contains(g.geom, h.geom);
                     """
cur.execute(spatial_join)
conn.commit()

# close cursor
cur.close()

# close connection
conn.close()

### How it works

The query is straightforward enough; we'll use the UPDATE standard SQL command to update the values in the name field of our table, geodata.pebble_beach_greens, with the hole numbers located in the pebble_beach_hole_num table.

We follow up by setting the name value from our geodata.pebble_beach_hole_num table, where the field name also exists and holds our needed attribute values.

Our WHERE clause uses the PostGIS query, ST_Contains, to return True if the point lies inside our greens, and if so, it will update our values.

This was easy and demonstrates the great power of spatial relationships.

## Conducting a complex spatial analysis query using ST_Distance()

